In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
from contextlib import contextmanager
from sklearn.model_selection import train_test_split

# Set style for plots
style.use('ggplot')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# https://www.sciencedirect.com/science/article/pii/S0167923609001377
# http://www3.dsi.uminho.pt/pcortez/wine5.pdf
wine_quality = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
print("Dataset length", len(wine_quality))
wine_quality.head()

## ¿Qué variables tenemos?  

Lo primero que hay que hacer es tratar de identificar el tipo de variables que tenemos a la mano, en algunos *datasets* esto es posible con tan solo leer los nombres de las columnas (como es el caso de el *dataset* con el que estamos trabajando). Sin embargo, hay algunos casos en los que los nombres no son provistos (o están ofuscados) por diversas razones.

Aprovechando que nuestro *dataset* si tiene nombres, podemos verlos con:

In [ ]:
wine_quality.columns

Probablemente si no eres un conocedor de vinos, las variables tengan poco o nada de sentido para ti.**Idealmente nosotros deberíamos tener conocimiento del tema sobre el que vamos a trabajar (o podemos conseguir un experto que nos guíe)**... pero por ahora pretendamos que sabemos lo que hacemos. Visita este link si quieres <a href="https://waterhouse.ucdavis.edu/whats-in-wine" target="_blank">una ligera introducción a los componentes del vino</a>.

Aquí hay una pequeña descripción de las variables:  

 - **fixed acidity**: *Acids are major wine constituents and contribute greatly to its taste. In fact, acids impart the sourness or tartness that is a fundamental feature in wine taste. In this case, this measures tartaric acid as $\frac{g(tartaric\ acid)}{dm^3}$.*
 - **volatile acidity**: *Volatile acidity refers to the steam distillable acids present in wine, in this case it measures acetic acid as $\frac{g(acetic\ acid)}{dm^3}$.*
 - **citric acid**: *Citric acid is often added to wines to increase acidity, complement a specific flavor or prevent ferric hazes. It can be added to finished wines to increase acidity and give a “fresh” flavor. Measure given in $\frac{g}{dm^3}$.*
 - **residual sugar**: *Residual sugar is from natural grape sugars leftover in a wine after the alcoholic fermentation finishes. Noticeably sweet wines start at around 35 grams per liter of residual sugar and then go up from there. Measure given in $\frac{g}{dm^3}$.*
 - **chlorides**: *In dry regions the, frequent irrigation increases soil salinity, and thus the salinity in wines. Measurement given in . Measure given in $\frac{g(sodium\ chloride)}{dm^3}$.*
 - **free sulfur dioxide**: *Free sulfur dioxide is a measure of the amount of $SO_2$ that is not bound to other molecules, and is used to calculate molecular $SO_2$. Measurements given in. Measure given in $\frac{mg}{dm^3}$.*
 - **total sulfur dioxide**: *Total Sulfur Dioxide ($TSO_2$) is the portion of $SO_2$ that is free in the wine plus the portion that is bound to other chemicals in the wine such as aldehydes, pigments, or sugars. Measure given in $\frac{mg}{dm^3}$.*
 - **density**: *The density of wine is primarily determined by the concentration of alcohol, sugar, glycerol, and other dissolved solids. Measure given in $\frac{g}{cm^3}$.*
 - **pH**: *Influences microbiological stability, affects the equilibrium of tartrate salts, determines the effectiveness of sulfur dioxide and enzyme additions, influences the solubility of proteins and effectiveness of bentonite and affects red wine colour and oxidative and browning reactions. Scale from 0 (very acidic) to $14$ (very basic); most wines are between $3$-$4$ on the pH scale.*
 - **sulphates**: *Sulfites comprise a range of sulfur compounds—particularly sulfur dioxide ($SO_2$)—that are a natural by-product of the fermentation process that work as a preservative against certain yeast and bacteria.*
 - **alcohol**: *The percent alcohol content of the wine.*


 - **quality**: *A score between $0$ and $10$ (based on sensory data).*





Sources: https://wineserver.ucdavis.edu/industry-info/enology/methods-and-techniques/common-chemical-reagents/citric-acid https://waterhouse.ucdavis.edu/whats-in-wine https://winefolly.com/deep-dive/what-is-residual-sugar-in-wine/ https://www.mt.com/gb/en/home/supportive_content/ana_chem_applications/titration/AP015.html https://www.laboratoire-obst.com/so2-libre-en.html 

## Antes de continuar ⚠️  

 > Si lo que estás esperando hacer con la información es crear un modelo predictivo, lo primero que hay que hacer es separar los datos en conjuntos de prueba, entrenamiento y, si puedes, validación. El análisis exploratorio de datos se debe conducir únicamente sobre los datos de entrenamiento, ya que realizar el análisis en todo el conjunto de datos nos llevaría a tomar decisiones teniendo en cuenta datos a los que, en teoría, tu modelo no tendría acceso en producción. Es decir, este es un problema de filtración de datos.
 
Teniendo esto en mente, vamos a separar nuestros datos con:

In [ ]:
wine_train, wine_test = train_test_split(wine_quality)

## Completitud en los datos  
Antes de comenzar cualquier análisis, es bueno revisar los datos para buscar información faltante; y en caso de que la haya, es nuestra tarea decidir qué es lo que podemos hacer con esos registros faltantes. Con los dataframes de `pandas` podemos usar `info` para encontrar los datos faltantes: También podríamos haber usado [missingno](https://github.com/ResidentMario/missingno) para tener una representación visual de esta información.

In [ ]:
wine_train.info()
wine_test.info()

Y pues no, no hay datos faltantes... sin embargo si faltaran, debes saber que existe toda una metodología para decidir cómo actuar ante datos faltantes en nuestro *dataset*. Pero de eso podemos hablar en otro momento.

## Estadísticas descriptivas  

El segundo paso a dar, es ver las estadísticas descriptivas de nuestra información, esto nos ayudará a darnos una idea de los posibles valores de nuestro dataset. El paquete `pandas` ofrece el método `describe` para obtener una vista detallada y completa de algunas de las estadísticas más comunes:

In [ ]:
wine_train.describe()

De este resumen estadístico, una de las primeras cosas que podrían resultar extrañas es que la variable **quality** únicamente toma valores entre $3$ y $8$ (a pesar de que en la descripción original dice que los valores van de $0$ a $10$. Lo cual representa un problema puesto que nuestro modelo predictivo no tendrá ejemplos de vinos con una calidad de $0$ o de $10$, por ejemplo. Pero por el momento, no nos vamos a preocupar mucho por eso.

También tengo que decir que algun conocedor del tema podría tener opiniones acerca de los rangos de valores que cubren ciertas variables... pero nosotros vamos a pasar al análisis gráfico.

## Análisis gráfico  

In [ ]:
@contextmanager
def plot(title=None, xlabel=None, ylabel=None, figsize=(9,5)):
    fig = plt.figure(figsize=figsize)
    ax = fig.gca()
    yield ax
    if title:
        ax.set_title(title)
    ax.set_xlabel(xlabel, size=15)
    ax.set_ylabel(ylabel, size=15)
    

### Histogramas  
Primero podemos echarle un ojo a la distribución de la variable `quality`, que como ya sabemos que es una variable discreta y que los valores van, en teoría de 0 a 10, podemos simplemente usar `countplot` del módulo *seaborn*:

In [ ]:
with plot(title="Counts of `qualuty`", xlabel="Quality", ylabel="Count") as ax:
    sns.countplot(x="quality", palette=("Accent"), data=wine_train, ax=ax)

Como puedes ver, nuestro conjunto de datos está desbalanceado, con muchos mas $5$ y $6$ que cualquier otro valor.

Ahora lo que podemos hacer es revisar algunas otras distribuciones, ahora sí, usando histogramas:

In [ ]:
with plot(title="Fixed Acidity distribution", xlabel="Acidity") as ax:
    sns.distplot(wine_train["volatile acidity"], ax=ax)

Podemos usar un poco de código para visualizar más de una variable a la vez:

In [ ]:
variables = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality']

columns = 4

fig, axes = plt.subplots(len(variables) //columns, columns, figsize=(15,8))

for current_idx, variable in enumerate(variables):
    i = current_idx // columns
    j = current_idx % columns
    sns.distplot(wine_train[variable], ax=axes[i][j])
    axes[i][j].set_title(variable)
    axes[i][j].set_xlabel("")
    
plt.tight_layout()

De esta gráfica podemos ver que muchas de las variables tienen una distribución asimétrica (`fixed acidity`, `residual sugar`, `chlorides`, por ejemplo), además de que al parecer algunos valores tienen valores extremos (`residual sugar`, `sulphates`, `total sulfur dioxide`). Tal vez merezcan más exploración...

### Boxplots  
Como mencioné anteriormente, existen algunas variables que merecen un poco más de exploración ya que parecen tener valores extremos, las *boxplots* nos permiten encontrar precisamente estos valores extremos. Es fácil graficar *boxplots* con *seaborn*.

In [ ]:
variables = ['fixed acidity', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 
             'total sulfur dioxide','sulphates', 'alcohol']

fig, axes = plt.subplots(1, len(variables), figsize=(15,6))

for ax, variable in zip(axes, variables):
    ax = sns.boxplot( y=variable, data=wine_train, ax=ax)
plt.tight_layout()

Como ya sabemos, los puntos fuera de las líneas horizontales son los famosos *outliers* o "valores atípicos", dependiendo de la aplicación podemos reaccionar de diversas maneras frente a ellos... a veces los *outliers* se eliminan, a veces se transforman, o a veces se dejan porque tienen alto valor predictivo.

### Scatterplots  
El siguiente paso es tratar de identificar relaciones entre variables, podríamos por ejemplo usar un *scatterplot* para ver qué tipo de relación existe entre la cantidad de alcohol y la calidad de un vino:

In [ ]:
with plot(title="Alcohol - Quality scatterplot", xlabel="Alcohol", ylabel="Quality") as ax:
    sns.scatterplot(x="alcohol", y="quality", data=wine_train, ax=ax) 

Tal vez esta gráfica no sea tan reveladora, ya que nuestra variable `quality` es más bien del tipo categórico y es dificil identificar una tendencia. Otra cosa a notar es que las correlaciones también se pueden y, en la mayoría de los casos, se deben identificar entre las variables independientes también, no solo entre una de ellas y la variable dependiente. Por ejemplo, entre `free sulfur dioxide` y `total sulfur dioxide`: 

In [ ]:
with plot(title="Free Sulfur Dioxide - Total Sulfur Dioxide scatterplot", xlabel="Free Sulfur Dioxide", ylabel="Total Sulfur Dioxide") as ax:
    sns.scatterplot(x="free sulfur dioxide", y="total sulfur dioxide", data=wine_train, ax=ax) 

En esta gráfica si se puede observar una clara relación entre las variables, ¿cierto?

Si quieres ver solamente el grado de de correlación (sin necesidad de tratar de encontrarlo tu mismo desde una *scatterplot*) también podemos hacer uso de las matrices de correlación.

### Matrices de correlación  
Una matriz de correlación no es más que una matriz de números (cada número va de -1 a 1) que nos indican qué tan relacionadas están una variable con otra. Existen [3 métodos](https://datascience.stackexchange.com/a/64261) para calcular esta correlación. Para calcularla en nuestro dataframe de vinos, podemos simplemente usar el método `corr` de un *dataframe*:

In [ ]:
correlation = wine_train.corr(method="pearson")
correlation.head()

Luego, para graficar estos números podemos usar un `heatmap` de *seaborn*:

In [ ]:
with plot(title="Free Sulfur Dioxide - Total Sulfur Dioxide scatterplot", xlabel="Free Sulfur Dioxide", ylabel="Total Sulfur Dioxide") as ax:
    sns.heatmap(correlation, vmin=-1,cmap= 'coolwarm',  ax=ax) 